# Calculation of association between HFE variants and *Yersinia pestis* status

## Purpose of the notebook

This notebook estimates the association between selected HFE variants and plague case–control status in Early Medieval individuals (and, if applicable, comparison cohorts). It is designed to be fully reproducible for peer reviewers and readers.

The workflow:

1. Loads an input CSV containing sample identifiers, plague status, and HFE genotypes (e.g. C282Y, H63D).
2. Performs basic quality-control and filtering of samples and variants (as specified in the Methods).
3. Computes allele counts and genotype frequencies in cases and controls.
4. Fits association models (e.g. Fisher’s exact test and/or logistic regression) to estimate effect sizes, odds ratios, confidence intervals, and *P*-values.
5. Applies multiple-testing correction where appropriate.
6. Exports summary tables for use in the manuscript and Supplementary Information.

---

## System requirements

- Operating system: Linux (Ubuntu 20.04+), macOS 12+, or Windows 10+
- Programming language: **Python 3.11**
- Hardware: Standard 64-bit desktop or laptop; ≥ 8 GB RAM recommended

---

## Required Python packages

The notebook assumes the following Python packages are installed:

- `pandas` (data handling)
- `numpy` (numerical computations)
- `scipy` (statistical tests)
- `statsmodels` (regression / modelling)
- `matplotlib` (optional: plots and visualisation)
- `jupyterlab` or `notebook` (to run this file)

Exact versions and full dependency list are provided in the project’s `requirements.txt` or `environment.yml` file.

---

## Expected inputs and outputs

### Input

A CSV file containing, at minimum:

- `Sample_ID` – unique identifier per individual  
- A case–control indicator (e.g. `Plague_status` with values `"case"` / `"control"` or equivalent)  
- One or more HFE genotype columns, for example:  
  - `HFE_C282Y`  
  - `HFE_H63D`  

Genotypes may be coded as 0/1/2 (dosage of the effect allele) or as alleles (e.g. `GG`, `GA`, `AA`), as described in the Methods section of the manuscript.

The default path to the input file is set in a configuration cell (e.g. `INPUT_CSV = "HFE_input.csv"`); this can be modified by the user as needed.

### Outputs

The notebook writes one or more summary tables to disk, for example:

- `HFE_case_control_summary.csv` – allele/genotype counts and frequencies in cases vs. controls  
- `HFE_association_results.csv` – effect sizes, odds ratios, confidence intervals, raw and adjusted *P*-values  

Optional:

- Plots (e.g. forest plots or barplots) saved as `.png` or `.pdf` files in a designated output folder (e.g. `plots/`).

---

## Statistical methods (summary)

- Association between HFE variants and plague status is evaluated using:
  - 2×2 (or 2×3) contingency tables with Fisher’s exact test and/or chi-square tests, and/or  
  - Logistic regression models with plague status as the outcome and HFE genotype as the predictor, optionally adjusted for covariates (e.g. age, sex, ancestry PCs) if present.
- Effect sizes are reported as odds ratios (OR) with 95% confidence intervals.
- Multiple-testing correction (e.g. Benjamini–Hochberg FDR) is applied if more than one variant/test is evaluated.

Full details are given in the manuscript and Supplementary Methods.

---

## How to run

1. **Set up the environment**

   Create and activate the analysis environment (example using conda):
  
  ```bash
   conda env create -f environment.yml
   conda activate hfe-plague
   ```
   
   or install packages directly with pip:
   
   ```bash
   pip install -r requirements.txt
   ```

2. **Open the notebook**
    
    Launch Jupyter and open this file (e.g. 'HFE_Analysis.ipynb')
    
     ```bash
    jupyter lab
     ```
    or
     ```bash
    jupyter notebook
     ```
3. **Configure the input file**

    In the configuration cell, ensure the path to the input CSV is correct, for example:
    
    INPUT_CSV = "HFE_input.csv"  # update if needed

4. **Run the analysis**

    In the Jupyter interface, select:
    
    - 'Kernel' -> 'Restart & Run All'
    
    All cells will execute in sequence, and the output tables and any plots will be written to the working directory.
    
5. **Expected runtime**
    
    On a standard desktop or laptop, the complete analysis typically runs in < 1 minute for the datasets used in the manuscript.



In [1]:
# BLOCK 1 — Setup, load data, and helper functions
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact, chi2_contingency, norm

# --- Load the UPDATED CSV ---
path = "../../../../../Google Drive/My Drive/00_Scheib_working/0. Anglo-Saxons | Scheib | 2026/HLA/HFE_input.csv"
df = pd.read_csv(path)
df.columns = [c.strip() for c in df.columns]

# Normalize labels
df["Case/Control"] = df["Case/Control"].astype(str).str.strip().str.lower()

# --- Genotype utilities ---
VALID_GTS = {"0/0","0|0","0/1","1/0","0|1","1|0","1/1","1|1"}
CARRIER_GTS = {"0/1","1/0","0|1","1|0","1/1","1|1"}

def gt_is_valid(x):
    return str(x).strip() in VALID_GTS

def gt_to_ref_alt(x):
    x = str(x).strip()
    if x in {"0/0","0|0"}: return (2,0)
    if x in {"0/1","1/0","0|1","1|0"}: return (1,1)
    if x in {"1/1","1|1"}: return (0,2)
    return (np.nan, np.nan)

def allele_counts(series: pd.Series):
    ref = alt = n = 0
    for v in series.dropna():
        if gt_is_valid(v):
            r,a = gt_to_ref_alt(v)
            ref += r
            alt += a
            n += 1
    total = ref + alt
    af_alt = (alt/total) if total else np.nan
    return n, ref, alt, total, af_alt

def dominant_carrier_counts(series: pd.Series):
    total = carriers = noncarriers = 0
    for v in series.dropna():
        v = str(v).strip()
        if v in VALID_GTS:
            total += 1
            if v in CARRIER_GTS:
                carriers += 1
            else:
                noncarriers += 1
    return total, carriers, noncarriers

def odds_ratio_and_ci(a,b,c,d, alpha=0.05):
    """
    Compute OR and Wald 95% CI with Haldane-Anscombe correction if any zero cell.
    a = cases exposed (carriers or alt alleles)
    b = cases unexposed (non-carriers or ref alleles)
    c = controls exposed
    d = controls unexposed
    """
    a0,b0,c0,d0 = a, b, c, d
    if min(a,b,c,d) == 0:
        a, b, c, d = a+0.5, b+0.5, c+0.5, d+0.5
    OR = (a*d) / (b*c)
    se = np.sqrt(1/a + 1/b + 1/c + 1/d)
    z = norm.ppf(1 - alpha/2)
    lcl = np.exp(np.log(OR) - z*se)
    ucl = np.exp(np.log(OR) + z*se)
    return float(OR), float(lcl), float(ucl), (a0,b0,c0,d0)


In [3]:
# BLOCK 2 — Dominant (carrier) model: carrier frequencies, ORs with 95% CIs, Fisher's exact p
import numpy as np
from scipy.stats import fisher_exact

variants = ["C282Y","H63D"]

def run_dominant_block(df_sub: pd.DataFrame, subset_name: str):
    rows = []
    for var in variants:
        # Cases vs controls
        cases = df_sub[df_sub["Case/Control"]=="case"]
        ctrls = df_sub[df_sub["Case/Control"]=="control"]
        case_total, case_car, case_non = dominant_carrier_counts(cases[var])
        ctrl_total, ctrl_car, ctrl_non = dominant_carrier_counts(ctrls[var])
        
        # 2x2 table
        a, b, c, d = case_car, case_non, ctrl_car, ctrl_non
        
        # Fisher's exact p (two-sided)
        table = np.array([[a,b],[c,d]], dtype=float)
        try:
            fisher_or, fisher_p = fisher_exact(table, alternative="two-sided")
        except Exception:
            fisher_or, fisher_p = (np.nan, np.nan)
        
        # OR and 95% CI (Haldane-corrected if zeros)
        or_est, lcl, ucl, raw_cells = odds_ratio_and_ci(a,b,c,d, alpha=0.05)
        
        rows.append({
            "Subset": subset_name,
            "Variant": var,
            "Cases_total": case_total,
            "Cases_carriers": case_car,
            "Cases_carrier_freq": (case_car/case_total) if case_total else np.nan,
            "Controls_total": ctrl_total,
            "Controls_carriers": ctrl_car,
            "Controls_carrier_freq": (ctrl_car/ctrl_total) if ctrl_total else np.nan,
            "OR_dominant": or_est,
            "CI95_lower": lcl,
            "CI95_upper": ucl,
            "Fisher_P": fisher_p,
            "Cells[a,b,c,d]": str(raw_cells),
        })
    return pd.DataFrame(rows)

# Subsets
edi_df = df[df["ID"].astype(str).str.startswith("EDI")].reset_index(drop=True)
all_df = df.copy()

dom_edi = run_dominant_block(edi_df, "EDI-only")
dom_all = run_dominant_block(all_df, "All samples")
dom = pd.concat([dom_edi, dom_all], ignore_index=True)

# Save
dom.to_csv("HFE_dominant_model_results.csv", index=False)
dom


,Subset,Variant,Cases_total,Cases_carriers,Cases_carrier_freq,Controls_total,Controls_carriers,Controls_carrier_freq,OR_dominant,CI95_lower,CI95_upper,Fisher_P,"Cells[a,b,c,d]"
0,EDI-only,C282Y,9,3,0.333333,25,4,0.160000,2.625,0.455958,15.112396,0.348173,"(3, 6, 4, 21)"
1,EDI-only,H63D,9,3,0.333333,25,5,0.200000,2.000,0.366326,10.919234,0.648847,"(3, 6, 5, 20)"
2,All samples,C282Y,9,3,0.333333,86,10,0.116279,3.800,0.818946,17.632417,0.103885,"(3, 6, 10, 76)"
3,All samples,H63D,9,3,0.333333,86,25,0.290698,1.220,0.282770,5.263641,0.720230,"(3, 6, 25, 61)"


In [4]:
# BLOCK 3 — Allele-based analysis with 95% CI added to odds ratios
from scipy.stats import fisher_exact, chi2_contingency

variants = ["C282Y","H63D"]

def run_allele_block(df_sub: pd.DataFrame, subset_name: str):
    out_rows = []
    for var in variants:
        cases = df_sub[df_sub["Case/Control"]=="case"]
        ctrls = df_sub[df_sub["Case/Control"]=="control"]
        n_case, r_case, a_case, tot_case, af_case = allele_counts(cases[var])
        n_ctrl, r_ctrl, a_ctrl, tot_ctrl, af_ctrl = allele_counts(ctrls[var])
        
        # 2x2 (alleles)
        a, b, c, d = a_case, r_case, a_ctrl, r_ctrl
        
        # Fisher and chi-square
        table = np.array([[a,b],[c,d]], dtype=float)
        try:
            fisher_or, fisher_p = fisher_exact(table, alternative="two-sided")
        except Exception:
            fisher_or, fisher_p = (np.nan, np.nan)
        try:
            chi2, chi2_p, _, _ = chi2_contingency(table, correction=False)
        except Exception:
            chi2, chi2_p = (np.nan, np.nan)
        
        # OR & 95% CI
        or_est, lcl, ucl, raw_cells = odds_ratio_and_ci(a,b,c,d, alpha=0.05)
        
        out_rows.append({
            "Subset": subset_name,
            "Variant": var,
            "Case_N": n_case,
            "Case_Alt": a_case,
            "Case_Ref": r_case,
            "Case_Alt_AF": af_case,
            "Control_N": n_ctrl,
            "Control_Alt": a_ctrl,
            "Control_Ref": r_ctrl,
            "Control_Alt_AF": af_ctrl,
            "OR_allele": or_est,
            "CI95_lower": lcl,
            "CI95_upper": ucl,
            "Fisher_P": fisher_p,
            "Chi2_Stat": chi2,
            "Chi2_P": chi2_p,
            "Cells[a,b,c,d]": str(raw_cells),
        })
    return pd.DataFrame(out_rows)

edi_df = df[df["ID"].astype(str).str.startswith("EDI")].reset_index(drop=True)
all_df = df.copy()

allele_edi = run_allele_block(edi_df, "EDI-only")
allele_all = run_allele_block(all_df, "All samples")
allele_results = pd.concat([allele_edi, allele_all], ignore_index=True)

# Save
allele_results.to_csv("HFE_allele_results_with_CI.csv", index=False)
allele_results


,Subset,Variant,Case_N,Case_Alt,Case_Ref,Case_Alt_AF,Control_N,Control_Alt,Control_Ref,Control_Alt_AF,OR_allele,CI95_lower,CI95_upper,Fisher_P,Chi2_Stat,Chi2_P,"Cells[a,b,c,d]"
0,EDI-only,C282Y,9,3,15,0.166667,25,4,46,0.080000,2.300000,0.461409,11.464893,0.370564,1.076534,0.299474,"(3, 15, 4, 46)"
1,EDI-only,H63D,9,3,15,0.166667,25,6,44,0.120000,1.466667,0.325722,6.604133,0.689889,0.250998,0.616373,"(3, 15, 6, 44)"
2,All samples,C282Y,9,3,15,0.166667,86,10,162,0.058140,3.240000,0.803435,13.065901,0.111649,3.011027,0.082700,"(3, 15, 10, 162)"
3,All samples,H63D,9,3,15,0.166667,86,27,145,0.156977,1.074074,0.291000,3.964384,1.000000,0.011507,0.914575,"(3, 15, 27, 145)"
